In [32]:
# Load modules

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

In [50]:
trade_bra = pd.read_csv('data/econ_data/trade_bra.csv', dtype = {'HS ID 4': str}).dropna(subset = ['Exports', 'Imports'])
wages_bra = pd.read_csv('data/econ_data/wages_bra.csv').dropna(subset = ['Total Jobs', 'Total Yearly Wages'])
trade_chi = pd.read_csv('data/econ_data/trade_chi.csv', dtype = {'HS ID 4': str}).dropna(subset = ['Exports', 'Imports'])
prod_chi = pd.read_csv('data/econ_data/wages_output_chi.csv')#.dropna(subset = ['Total Jobs', 'Total Yearly Wages'])
# prod_chi = pd.read_csv('data/econ_data/sources/production_USD.csv')#.dropna(subset = ['Total Jobs', 'Total Yearly Wages'])
# wages_bra_raw = wages_bra_raw.sample(frac = .)

/home/miglesia/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
def filter_small_cats(df, level, cat = 'BRA ID MIR', col = 'Total Yearly Wages'):
    grouped = df.groupby(cat).sum().sort_values(by = col, ascending = False).dropna()
    cumsum = grouped.div(grouped.sum()).cumsum()
    label_sel = cumsum.loc[cumsum[col] < level].index.values
    df_ = df.loc[df[cat].isin(label_sel)]
    return df_

df1 = filter_small_cats(wages_bra, .98, 'BRA ID MIR')
df2 = filter_small_cats(df1, .98, 'CNAE ID C')
df3 = filter_small_cats(df2, .98, 'CBO ID F')

# for df in [df1, df2, df3]:
#     print np.true_divide(len(df),len(wages_bra))
#     print df['Total Yearly Wages'].sum()/wages_bra['Total Yearly Wages'].sum()

wages_bra = df3

In [5]:
# wages_bra['Total Yearly Wages'].sum()/x['Total Yearly Wages'].sum()

## Info of aggregations

In [45]:
# Load info
path = 'data/disagg_struct_refs/formatted/'

CIIU = pd.read_csv(path+'CIIU_full.csv')
CBO = pd.read_csv(path+'CBO_full.csv').astype(str)
CNAE = pd.read_csv(path+'CNAE_full.csv').astype(str)
HS = pd.read_csv(path+'hs_full.csv').astype(str)

#Choose columns
CIIU_agg_ref = CIIU[['CIIU ID 4', 'CIIU ID 2', 'CIIU ID 1', 'CIIU ID 0']]
CNAE_agg_ref = CNAE[['CNAE ID C', 'CNAE ID G', 'CNAE ID D', 'CNAE ID S','CNAE ID 0']]
CBO_agg_ref = CBO[['CBO ID F', 'CBO ID SG', 'CBO ID SGP', 'CBO ID GG', 'CBO ID 0']]
HS_agg_ref = HS[['HS ID 4', 'HS ID 2']]

In [31]:
#Geographic info

geo_ids = pd.concat([wages_bra['BRA ID MIR'], trade_bra['BRA ID MIR']]).unique()
GEO_bra = pd.DataFrame([[gid[:3],gid[:5],gid]  for gid in geo_ids], columns = ['BRA ID ST', 'BRA ID MER', 'BRA ID MIR'])
# GEO_bra.to_csv('data/Classifications/GEO_bra.csv', index = False)

GEO_chi = pd.read_csv(path+'GEO_chi.csv')[['reg_id', 'prov_id', 'comuna_id']]
GEO_chi = GEO_chi.rename({'reg_id':'CHI ID REG', 'prov_id':'CHI ID PRO', 'comuna_id':'CHI ID COM'},axis = 1)


NameError: name 'wages_bra' is not defined

In [ ]:
# Aggregate across 4 years

trade_bra_4y = trade_bra.groupby([u'BRA ID MIR', u'HS ID 4']).sum().reset_index()
wages_bra_4y = wages_bra.groupby([ u'BRA ID MIR',u'CNAE ID C', u'CBO ID F']).sum().reset_index()


In [ ]:
from decimal import Decimal

In [ ]:
Geo_lev = 'MIR'

trade_bra_4y_gl = trade_bra_4y.merge(GEO_bra).groupby(['BRA ID '+Geo_lev, 'HS ID 4']).sum().reset_index()
wages_bra_4y_gl = wages_bra_4y.merge(GEO_bra).groupby(['BRA ID '+Geo_lev, 'CNAE ID C', 'CBO ID F']).sum().reset_index()

nodes_df_list = []
df_list =[]
confs = [('0', '0', '2', '2'),
         ('0', 'S', '2', '2'),
         ('GG', '0', '2', '2'),
         ('GG', 'S', '2', '2'),
        ('0', 'G', '4', '2')]
name_list = []
for conf in confs:
    (CBO_lev, CNAE_lev, HS_Exp, HS_Imp) = conf
    
    col_name = 'corr_'+Geo_lev+'_'+'_'.join([c for c in conf])
    name_list += [col_name]

    # Bring 3 datasets and give them node_names. Prepare for correlation

    x = wages_bra_4y_gl.merge(CBO_agg_ref).groupby(['BRA ID '+Geo_lev, 'CNAE ID C', 'CBO ID '+CBO_lev]).sum()[['Total Yearly Wages']].reset_index()
    print '%.2E' % Decimal(x['Total Yearly Wages'].sum())
    x = x.merge(CNAE_agg_ref).groupby(['BRA ID '+Geo_lev, 'CNAE ID '+CNAE_lev, 'CBO ID '+CBO_lev]).sum()[['Total Yearly Wages']].reset_index()
    x_ = x

    #     use label for naming nodes
    x_ = x.merge(CNAE[['CNAE ID '+CNAE_lev,'CNAE label '+CNAE_lev]].drop_duplicates())
    x_ = x_.merge(CBO[['CBO ID '+CBO_lev, 'CBO label '+CBO_lev]].drop_duplicates())
    x_['node_name'] = 'Wages in ' + x_[[col for col in x_.columns if 'label' in col]].apply(' of '.join, axis=1)
    # use ID for naming nodes
    #     x_['node_name'] = 'Wages in ' + x_[[col for col in x_.columns if 'label' in col and 'BRA' not in col]].apply(' of '.join, axis=1)

    x_ = x_.drop([col for col in x_.columns if 'label' in col], axis = 1)
    
    
    #node size
    x_ = x_.merge(x.groupby(['CNAE ID '+CNAE_lev,'CBO ID '+CBO_lev]).sum().reset_index().rename({'Total Yearly Wages': 'node_value_USD'}, axis = 1))
    x = x_

    if CNAE_lev != '0':
        x = x.merge(CNAE[list(set(['CNAE ID '+CNAE_lev,'CNAE ID S']))].drop_duplicates())
    if CBO_lev != '0':
        x = x.merge(CBO[list(set(['CBO ID '+CBO_lev,'CBO ID GG']))].drop_duplicates())
    x = x.rename({'CNAE ID S': 'att_A', 'CBO ID GG': 'att_B', 'Total Yearly Wages': 'value_USD'}, axis = 1)
    print '%.2E' % Decimal(x['value_USD'].sum())


    # Trade in HS2

    ### Exports
    y = trade_bra_4y_gl.merge(HS[['HS ID 4', 'HS ID 2']].drop_duplicates()).groupby(['BRA ID '+Geo_lev, 'HS ID '+HS_Exp]).sum().reset_index()
    print '%.2E' % Decimal(y['Exports'].sum())
    y = y.drop([col for col in y.columns if 'Imp' in col], axis = 1)
    y_ = y.merge(HS[['HS ID '+HS_Exp,'HS label '+HS_Exp]].drop_duplicates())
    y_['node_name'] = 'Exports of ' + y_[[col for col in y_.columns if 'label' in col]].apply(' of '.join, axis=1)
    y_ = y_.drop([col for col in y_.columns if 'label' in col], axis = 1)

    
    node_val = y.groupby(['HS ID '+HS_Exp]).sum()[['Exports']].rename({'Exports': 'node_value_USD'}, axis = 1).reset_index()
    y_ = y_.merge(node_val, on = 'HS ID '+HS_Exp)

    y = y_
    y = y.rename({'HS ID '+HS_Exp: 'att_A', 'Exports': 'value_USD'}, axis = 1)
    y['att_B'] = y['att_A'] = 'Exports'
    
    print '%.2E' % Decimal(y['value_USD'].sum())
    
    ### Imports
    z = trade_bra_4y_gl.merge(HS[['HS ID 4', 'HS ID 2']].drop_duplicates()).groupby(['BRA ID '+Geo_lev, 'HS ID '+HS_Imp]).sum().reset_index()
    print '%.2E' % Decimal(z['Imports'].sum())
    z = z.drop([col for col in z.columns if 'Exp' in col], axis = 1)
    z_ = z.merge(HS[['HS ID '+HS_Imp,'HS label '+HS_Imp]].drop_duplicates())

    z_['node_name'] = 'Imports of ' + z_[[col for col in z_.columns if 'label' in col]].apply(' of '.join, axis=1)
    z_ = z_.drop([col for col in z_.columns if 'label' in col], axis = 1)
    
    node_val = z_.groupby(['HS ID '+HS_Imp]).sum()[['Imports']].rename({'Imports': 'node_value_USD'}, axis = 1).reset_index()
    z_ = z_.merge(node_val, on = 'HS ID '+HS_Imp)
 
    z = z_
    z = z.rename({'HS ID '+HS_Imp: 'att_A', 'Imports': 'value_USD'}, axis = 1)
    z['att_B'] = z['att_A'] = 'Imports'
    print '%.2E' % Decimal(z['value_USD'].sum())

    ###

    concat_data = pd.concat([x, y, z])



    #Data for correlation
    df = concat_data.groupby(['BRA ID '+Geo_lev, 'node_name']).sum()[['value_USD']]
    
    print df.sum()
    
    #Drop small categories
    th = .99
    cat_size = df.reset_index().groupby('node_name').sum()[['value_USD']].sort_values(by = 'value_USD', ascending = False)
    cat_cumsum = (cat_size/cat_size.sum()).cumsum()
    l = list(cat_cumsum.loc[cat_cumsum['value_USD'] < th].index)   
    df = df.reset_index().loc[df.reset_index().node_name.isin(l)].set_index(['BRA ID '+Geo_lev, 'node_name'])
    
    #Save nodes info
    nodes_df = concat_data.groupby('node_name').first()[['att_A', 'att_B', 'node_value_USD']]
    nodes_df = nodes_df.reset_index().loc[nodes_df.reset_index().node_name.isin(l)].set_index(['node_name'])
    nodes_df_list += [nodes_df]
    
    # (cat_size/cat_size.sum()).cumsum().reset_index().plot(y = 'value_USD')
    # plt.show()
    
    print df.sum()

    
    df = df.unstack()
    df.columns = df.columns.droplevel(0)


    # correlations
    robust_th = 5
    corr_geolevel_yr = np.log10(df).corr(min_periods = robust_th)
    del corr_geolevel_yr.index.name
    del corr_geolevel_yr.columns.name

    df_list += [pd.DataFrame(corr_geolevel_yr.stack(), columns = [col_name])]

    print 'done conf '+ str(conf)

In [ ]:
# #Data for correlation
# df = pd.concat([x, y, z]).groupby(['BRA ID '+Geo_lev, 'node_name']).sum()[['value_USD']]

# print df.sum()

# #Drop small categories
# th = .99
# cat_size = df.reset_index().groupby('node_name').sum()[['value_USD']].sort_values(by = 'value_USD', ascending = False)
# cat_cumsum = (cat_size/cat_size.sum()).cumsum()
# l = list(cat_cumsum.loc[cat_cumsum['value_USD'] < th].index)   
# df = df.reset_index().loc[df.reset_index().node_name.isin(l)].set_index(['BRA ID '+Geo_lev, 'node_name'])

# # (cat_size/cat_size.sum()).cumsum().reset_index().plot(y = 'value_USD')
# # plt.show()

# print df.sum()



In [ ]:

# df = df.unstack()
# df.columns = df.columns.droplevel(0)


# # # correlations
# # robust_th = 5
# # corr_geolevel_yr = np.log10(df).corr(min_periods = robust_th)
# # del corr_geolevel_yr.index.name
# # del corr_geolevel_yr.columns.name

# # df_list += [pd.DataFrame(corr_geolevel_yr.stack(), columns = [col_name])]

# # print 'done conf '+ str(conf)

In [ ]:
# pd.DataFrame(df.stack()).reset_index().nunique()

In [ ]:
# pd.DataFrame(corr_geolevel_yr.stack(), columns = [col_name]).reset_index().nunique()

In [ ]:
# CNAE_lev = 'S'

# x = wages_bra_4y_gl.merge(CBO_agg_ref).groupby(['BRA ID '+Geo_lev, 'CNAE ID C', 'CBO ID '+CBO_lev]).sum()[['Total Yearly Wages']].reset_index().sample(frac = 0.1)
# x = x.merge(CNAE_agg_ref).groupby(['BRA ID '+Geo_lev, 'CNAE ID '+CNAE_lev, 'CBO ID '+CBO_lev]).sum()[['Total Yearly Wages']].reset_index()

# x.groupby('CNAE ID S').sum()[['Total Yearly Wages']]

In [ ]:
# z = trade_bra_4y_gl.merge(GEO_bra).merge(HS[['HS ID 4', 'HS ID 2']].drop_duplicates()).groupby(['BRA ID '+Geo_lev, 'HS ID '+HS_Exp]).sum().reset_index()
# d = z.groupby(['HS ID '+HS_Imp]).sum().sort_values(by = 'Imports')
# d


### Resulting data == correlations!!! for network

In [ ]:
def demean_df(df):
    dm1 = df - df.mean()
    dm2 = (dm1.T - dm1.T.mean()).T
    
    demeaned_df = dm2
    
    return demeaned_df

## Network plots

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

G_list = []

In [ ]:
import networkx as nx

In [ ]:
Geo_lev = 'MIR'

# trade_bra_4y_gl = trade_bra_4y.merge(GEO_bra).groupby(['BRA ID '+Geo_lev, 'HS ID 4']).sum().reset_index()
# wages_bra_4y_gl = wages_bra_4y.merge(GEO_bra).groupby(['BRA ID '+Geo_lev, 'CNAE ID C', 'CBO ID F']).sum().reset_index()

for i in range(len(confs)):
    conf = confs[i]
    print conf

    dem_corr = demean_df(df_list[i].unstack())
    dem_corr.columns = dem_corr.columns.droplevel(0)

    ## standardizing after demeaning
    std_dem_corr = dem_corr.div(dem_corr.std(), axis = 0)    

    name = 'corr_'+Geo_lev+'_'+'_'.join([c for c in conf])

    df = pd.DataFrame(std_dem_corr.stack(), columns = [name])
    network_data = df[(df[name] > df[name].quantile(4/5.)) & (df[name] < df[name].quantile(.99))]
    network_data.index.names = ['label4_x', 'label_y']

    edges = network_data.reset_index().rename({name: 'weight'}, axis  =1)

    G = nx.from_pandas_dataframe(edges, 'label4_x', 'label_y', 'weight')
    col_name = 'corr_'+Geo_lev+'_'.join([c for c in confs[i]])
    nx.write_gexf(G, col_name+'.gexf')

    G_list += [G]
    # #Main component
    # grcc=sorted(nx.connected_component_subgraphs(G), key = len, reverse=True)
    # G0=grcc[0]

    nodes = nodes_df_list[i].loc[G.nodes()]

    #Perform Graph Drawing
    plt.figure(figsize=(8, 8))
    posi = nx.spring_layout(G, k=0.15)
    node_sizes = .001*np.sqrt(nodes['node_value_USD'].values)
    colors = nodes.att_A.rank(method = 'dense').values

    # 

    nx.draw(G, posi, node_size = node_sizes, alpha=.6, node_color= colors, edgelist = [])
    nx.draw_networkx_edges(G, posi, edge_color='.7', alpha=.3)
    #     plt.savefig('../DataViva/Figures/'+'_'+kw+'_'+name)
    plt.show()

### nodes DataFrame

In [ ]:
import pandas as pd

### Nodes data

In [ ]:
j = 4

### Create and save node data

nodes = nodes_df_list[j]

value = nodes['node_value_USD']
# nodes['node_size'] = np.sqrt((value/value.median()).values)
nodes['node_size'] = 10 * np.power((value/value.median()).values, 1/4.)

nodes['color_code'] = nodes.att_A.rank(method = 'dense')

import matplotlib

x = plt.cm.get_cmap('rainbow', len(nodes.color_code.unique()))

colors_hex = []
for i in range(x.N):
    rgb = x(i)[:3] # will return rgba, we take only first 3 so we get rgb
    colors_hex += [matplotlib.colors.rgb2hex(rgb)]

colors_hex = [x.encode('UTF8') for x in colors_hex]


code_hex = pd.DataFrame([nodes.color_code.unique(), colors_hex], index = ['color_code', 'color']).T
nodes = nodes.reset_index().merge(code_hex)

nodes.to_csv(name_list[j]+'_nodes.csv')


In [ ]:
# #Load node data
# for j in range(len(name_list)):
#     nodes = pd.read_csv(name_list[j]+'_nodes.csv').rename({'Unnamed: 0': 'index'}, axis = 1)


### Edges data

In [ ]:

dem_corr = demean_df(df_list[j].unstack())
dem_corr.columns = dem_corr.columns.droplevel(0)

## standardizing after demeaning
std_dem_corr = dem_corr.div(dem_corr.std(), axis = 0)    

name = 'corr_'+Geo_lev+'_'+'_'.join([c for c in conf])

df = pd.DataFrame(std_dem_corr.stack(), columns = [name])
network_data = df[(df[name] > df[name].quantile(4/5.)) & (df[name] < df[name].quantile(.99))]
network_data.index.names = ['label_x', 'label_y']

edges = network_data.reset_index().rename({name: 'weight'}, axis  =1)

# G = nx.from_pandas_dataframe(edges, 'label_x', 'label_y', 'weight')
    

In [ ]:
# import sys    
# reload(sys)  
# sys.setdefaultencoding('utf8')

In [ ]:
# nodes.sample(30).sort_values(by = 'node_size')
# df_list[j].reset_index().nunique()

In [ ]:
import math

from bokeh.io import show, output_file
from bokeh.plotting import figure
from bokeh.models import GraphRenderer, StaticLayoutProvider, Circle, MultiLine, NodesOnly
from bokeh.models.graphs import from_networkx
from bokeh.models.tools import HoverTool



N = len(nodes)
node_indices = list(range(N))
# sizes = np.linspace(10, 20, N)
# node_sizes = np.sqrt(nodes['node_size'].values)


plot = figure(title="Networkx Integration Demonstration", x_range=(-1.1,1.1), y_range=(-1.1,1.1))

#####

graph = from_networkx(G, nx.spring_layout, center=(0,0), k = .15)

graph.node_renderer.data_source.add(list(nodes['color'].values), 'color')
graph.node_renderer.data_source.add(list(nodes['node_name'].values), 'node_name')
graph.node_renderer.data_source.add(list(nodes['node_size'].values), 'node_size')

graph.node_renderer.glyph = Circle(size='node_size', fill_color='color')
graph.node_renderer.selection_glyph = Circle(size=15, fill_color='#808000')
graph.node_renderer.hover_glyph = Circle(size=15, fill_color='#808000')

graph.edge_renderer.glyph = MultiLine(line_color="#CCCCCC", line_alpha=0.2, line_width=1)
graph.edge_renderer.selection_glyph = MultiLine(line_color='#808000', line_width=5)
graph.edge_renderer.hover_glyph = MultiLine(line_color='#808000', line_width=5)
graph.selection_policy = NodesOnly()

plot.renderers.append(graph)

hover = HoverTool(plot=plot, tooltips=dict(act="@node_name", value="@node_size"))
# hover = HoverTool(tooltips = [('Info', '@node_name'),('(x,y)', '($x, $y)')])

plot.add_tools(hover)

output_file("networkx_graph.html")
# output_notebook()
show(plot)

In [ ]:
CNAE[['CNAE ID S', 'CNAE label S']].drop_duplicates()

In [ ]:
CBO[['CBO ID GG', 'CBO label GG']].drop_duplicates()

In [ ]:
### PLOT CORR VALUES

for i in range(4):

    rk = pd.DataFrame(df_list[i].reset_index().groupby('level_0').mean().rank())
    rk.columns = ['rank']
    
    dem_corr = demean_df(df_list[i].unstack())
    dem_corr.columns = dem_corr.columns.droplevel(0)

    ## standardizing after demeaning
    std_dem_corr = dem_corr.div(dem_corr.std(), axis = 0)   

    df_corr = df_list[i].reset_index().merge(rk.reset_index())
    df_corr.columns = [u'level_0', u'level_1', 0, u'rank']
    df_std_dem = std_dem_corr.stack().reset_index().merge(rk.reset_index())


    dfs = [df_corr, df_std_dem]
    fig, axs = plt.subplots(1, 2, figsize = (10, 4))
    for i in range(2):
        df = dfs[i]
        ax = axs[i]
        df.sample(10000, replace = True).plot(x = 0, y = 'rank', marker = '.', linestyle = 'None', alpha = .1, ax = ax, color = '.5')

        if i == 0:
            ax.set_xlim(-.5, 1)
            ax.set_title('Pearson corr values \n for all nodes, ' +name)
            hilite = df_corr.loc[df_std_dem[0] > np.percentile(df_std_dem[0], 100 * 4/5.)]
            hilite.sample(3000, replace = True).plot(x = 0, y = 'rank', marker = '.', linestyle = 'None', ax = ax, alpha = .1)


        if i ==1:
            ax.axvline(x = df[0].quantile(4/5.), linestyle = '-', color = 'b')
            ax.axvline(x = df[0].quantile(.99), linestyle = '-', color = 'b')
            ax.set_xlim(-4, 4)
            ax.set_title('Demeaned and standardized \n corr values, '+name)
#             hilite = df_std_dem.loc[df_std_dem[0] > np.percentile(df_std_dem[0], 100 * 4/5.)]
#             hilite.sample(1000, replace = True).plot(x = 0, y = 'rank', marker = '.', linestyle = 'None', ax = ax)

    plt.show()



In [ ]:
#Node size info (maybe should be done when the node labels are set)

df = trade_bra.merge(HS[['HS ID 2', 'HS ID 4']].drop_duplicates()).groupby('HS ID 2').sum()
nds_y = nodes_df.loc[[row for row in nodes_df.index if 'Exports' in row]].reset_index().merge(df[['Exports']].reset_index(), left_on = 'att_A', right_on = 'HS ID 2').drop('HS ID 2', axis = 1).rename({'Exports' : 'value_USD'}, axis = 1)
nds_z = nodes_df.loc[[row for row in nodes_df.index if 'Imports' in row]].reset_index().merge(df[['Imports']].reset_index(), left_on = 'att_A', right_on = 'HS ID 2').drop('HS ID 2', axis = 1).rename({'Imports' : 'value_USD'}, axis = 1)

# df = wages_bra.merge(HS['CNAE ID '+CNAE_lev, 'CBO ID '+CBO_lev].drop_duplicates()).groupby(['CNAE ID '+CNAE_lev, 'CBO ID '+CBO_lev]).sum()

nds_x = nodes_df.loc[[row for row in nodes_df.index if 'Wages' in row]].reset_index()
nds_x['value_USD'] = 1762904178 #shortcut

nodes_df = pd.concat([nds_x, nds_y, nds_z]).set_index('node_name')

# colors...
# nodes_df.merge(CNAE[['CNAE ID D', 'CNAE ID S']], left_on = 'att_A', right_on = 'CNAE ID D', how = 'left').sample(30)#.loc[G.nodes()]

## Other

In [ ]:
# pd.concat(df_list, axis = 1).to_csv('data/Corr/corr_MER_S_GG_2_2', index = False)
# pd.concat(df_list, axis = 1).to_csv('data/Corr/corr_MER_S_GG_2_2_i')

In [ ]:
df = pd.read_csv('data/Corr/corr_MER_S_GG_2_2')

In [ ]:
import seaborn as sns; sns.set(style="ticks", color_codes=True)
pp = sns.pairplot(df.sample(10000).dropna(), diag_kind="kde", markers="+",
             plot_kws=dict(s=50, edgecolor="b", linewidth=1, alpha = .01),
             diag_kws=dict(shade=True))
[[ax.set_ylim(-.5,1) for ax in ax_list] for ax_list in pp.axes]
x = [-.5, 1]
[[ax.plot(x,x, c = '.5', linestyle = '--') for ax in ax_list] for ax_list in pp.axes]

plt.show()